In [1]:
import torch
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl

from data_loader_torch import SequenceLoader
#from network_simple import SimpleConvRecurrent

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy

plt.rcParams['image.cmap']= 'binary'
#plt.rcParams['image.cmap']= 'jet'

In [2]:
model = "unet"
tool = "pytorch"
data = "scin"
seq_length_min = int(3*60*24)
step_min = 30
window_train = int(2*60*24/step_min)
window_predict = int(1*60*24/step_min)
batch_size = 50
to_fit = False

In [3]:
print(f"Sequence length: {seq_length_min}")
print(f"Step: {step_min}")
print(f"Window train: {window_train}")
print(f"Window predict: {window_predict}")

Sequence length: 4320
Step: 30
Window train: 96
Window predict: 48


In [4]:
seq_validation = SequenceLoader(name='validation',
                                path_files='/mnt/data/resized_type_1',
                                seq_length_min=seq_length_min,
                                step_min=step_min,
                                window_train=window_train,
                                window_predict=window_predict,
                                data='scin',
                                station='summer')

In [5]:
seq_validation.__len__()

1000

In [9]:
seq = seq_validation.load(0)
X, y = seq

print(X.shape)

values = []
for i in range(0, 72):
    for j in range(0, 72):
        np_X = X[:,:, i, j].view(-1).detach().numpy()
        values.append((np.max(np_X), i, j))
            
def myFunc(x):
    return x[0] 

values.sort(reverse=True, key=myFunc)

torch.Size([96, 1, 72, 72])


In [10]:
values

[(0.7698414, 29, 55),
 (0.64308304, 31, 58),
 (0.5587389, 28, 59),
 (0.53135175, 33, 57),
 (0.5280604, 33, 56),
 (0.49177024, 29, 56),
 (0.4578734, 32, 58),
 (0.4365679, 29, 54),
 (0.39267498, 30, 54),
 (0.38787028, 28, 60),
 (0.36887914, 30, 55),
 (0.3654468, 29, 60),
 (0.35244283, 30, 58),
 (0.3420552, 28, 56),
 (0.3403049, 31, 59),
 (0.33419767, 45, 66),
 (0.32950562, 46, 70),
 (0.325456, 33, 58),
 (0.32391435, 48, 71),
 (0.32352453, 47, 70),
 (0.32345888, 46, 69),
 (0.32221723, 46, 66),
 (0.322185, 47, 67),
 (0.32199195, 46, 67),
 (0.32142457, 47, 71),
 (0.32123184, 32, 57),
 (0.3208866, 45, 68),
 (0.3206818, 55, 71),
 (0.32060158, 45, 67),
 (0.31951207, 48, 68),
 (0.31932846, 47, 68),
 (0.31897897, 52, 71),
 (0.31833372, 48, 69),
 (0.31833163, 46, 68),
 (0.31819105, 47, 69),
 (0.31815863, 49, 69),
 (0.31806725, 31, 54),
 (0.3178438, 49, 70),
 (0.31773195, 50, 70),
 (0.31758934, 53, 70),
 (0.31756908, 49, 71),
 (0.31751478, 50, 71),
 (0.31747246, 51, 71),
 (0.31693795, 48, 70),
 (0

In [ ]:
net = SimpleConvRecurrent(1, act_cuda=False) 

path = f'./results/state_dict_{model}_{window_train}_{window_predict}_{tool}_{data}.pth'

# original saved file with DataParallel
state_dict = torch.load(path, map_location=torch.device('cpu'))

# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

# load params
net.load_state_dict(new_state_dict)

In [ ]:
seq = seq_validation.load(0)
X, y = seq

predict = net.forward(X.view(1, window_train, 1, 72, 72), prediction_len=window_predict)

np_predict = predict.view(window_predict, 1, 72, 72).detach().numpy()
np_y = y.detach().numpy()

In [ ]:
a = 28
b = 35

plt.plot(np_y[:, 0, a, b])
plt.plot(np_predict[:, 0, a, b])

In [ ]:
x = np.linspace(240, 330, 72)
y = np.linspace(-60, 30, 72)

In [ ]:
np.max(np_predict)

In [ ]:
np.mean(np_y)

In [ ]:
i = 0

norm = mpl.colors.Normalize(vmin=.0, vmax=1.5)

# plot
ccrs.PlateCarree()
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(18, 8.5), subplot_kw={'projection': ccrs.PlateCarree()})
ax1, ax2 = axes

cp = ax1.contourf(x, y, np_y[i, 0, :], vmin=.0, vmax=1.5)
ax1.coastlines()
ax1.add_feature(cfeature.BORDERS, linestyle=':')
ax1.set_title('original')



cp = ax2.contourf(x, y, np_predict[i, 0, :], vmin=.0, vmax=1.5)
ax2.coastlines()
ax2.add_feature(cfeature.BORDERS, linestyle=':')
ax2.set_title('predict')

fig.colorbar(mpl.cm.ScalarMappable(norm=norm), ax=axes.ravel().tolist())

In [ ]:
size_seq = seq_validation.__len__()

l1loss_sum = []
l1loss_mean = []
l2loss_sum = []
l2loss_mean = []

for i in range(0, size_seq):
    seq = seq_validation.load(0)
    X, y = seq

    predict = net.forward(X.view(1, window_train, 1, 72, 72), prediction_len=window_predict)

    np_predict = predict.view(window_predict, 1, 72, 72).detach().numpy()
    np_y = y.detach().numpy()

    result = np_y-np_predict
    
    l1loss_mean.append(np.mean(np.abs(result)))
    l1loss_sum.apend(np.sum(np.abs(result)))

    l2loss_mean.apend(np.mean(result**2.0))
    l2loss_sum.apend(np.sum(result**2.0))

In [ ]:
relative_max_error = []
for i in range(0, 72):
    for j in range(0, 72):
        relative_max_error.append(np.max(np.abs((np_y[:,0, i, j] - np_predict[:, 0, i, j]))))
        #relative_max_error.append(np.max(np.abs((np_y[:,0, i, j] - np_predict[:, 0, i, j])/np_y[:,0, i, j]))*100)
        
relative_max_error = np.array(relative_max_error).reshape(72, 72).astype(np.float32)

In [ ]:
len(relative_max_error)

In [ ]:
plt.rcParams['image.cmap']= 'jet'

ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(24, 24), subplot_kw={'projection': ccrs.PlateCarree()})
im = ax.imshow(relative_max_error)

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')

# We want to show all ticks...
ax.set_xticks(np.arange(0, 72))
ax.set_yticks(np.arange(0, 72))
# ... and label them with the respective list entries
ax.set_xticklabels(np.arange(0, 72))
ax.set_yticklabels(np.arange(0, 72))

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
#for i in range(0, 72):
#    for j in range(0, 72):
#        text = ax.text(j, i, relative_max_error[i, j],
#                       ha="center", va="center", color="w")
        
#lt.rcParams['image.cmap']= 'binary'


ax.set_title("")
fig.tight_layout()
plt.show()